In [15]:
# def data_reduced(tensor_data,factors,reduced_k,num_comp):
# #toDo
#     f0=factors[0][:,:reduced_k]
#     f1=factors[1][:,:num_comp]

#     new_data=tl.tenalg.mode_dot(tensor_data, f1.T, mode=1, transpose=False)
#     new_data=tl.tenalg.mode_dot(new_data, f0.T, mode=0, transpose=False)
#     print("data size is",new_data.shape)
#     n_dim=new_data.shape
#     new_data=new_data.reshape(n_dim[0]*n_dim[1],n_dim[2]).T
#     print("unfolded data size is",new_data.shape)
#     return new_data


In [ ]:

این برنامه را چون انرژی که استفاده میکنه کارا نیست رها میکنیم و با 
QR
 که دقیق تره ادامه میدیم


In [16]:
import pandas as pd
import numpy as np
import tensorly as tl
from tensorly.decomposition import tucker

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA
import xgboost as xg

In [18]:
def tensorizing(k,data):
    #this func transform the 2D data to a 3D tensor
    # first dimension is timeseries samples , second is features and third dimension is time
    no_target_data=data.drop(columns=['DEMAND'])
    tensor_data=np.zeros((k,no_target_data.shape[1],data.shape[0]-k+1))
    for i in range(data.shape[0]-k+1):
        tensor_data[:,:,i]=no_target_data.iloc[i:i+k].to_numpy()
    print ("tensorized data shape is",tensor_data.shape)
    return tensor_data

def hosvd(tensor_data):
    core, factors = tucker(tensor_data, rank=None,random_state=17)
    # print("core rank is",core.shape)
    return core,factors



def data_reduced(tensor_data,reduced_window_size,num_comp):
    #inputs:
    # tensor_data is the data which we mean to reduce from first and second dimension
    # reduced_window_size is size of first dimension of the output tensor
    # num_comp is number of features of output
    #output:
    # a 2D array which obtained by unfolding a tensor of size (reduced_window_size,num_comp,tensor_data.shape[2])
    core, factors = tucker(tensor_data, rank=[reduced_window_size,num_comp,tensor_data.shape[2]],random_state=17)
    f0=factors[0]
    f1=factors[1]
    # print("u1",f1)
    new_data=tl.tenalg.mode_dot(tensor_data, f1.T, mode=1, transpose=False)
    new_data=tl.tenalg.mode_dot(new_data, f0.T, mode=0, transpose=False)
    print("reduced data size is",new_data.shape)
    n_dim=new_data.shape
    new_data=new_data.reshape(n_dim[0]*n_dim[1],n_dim[2]).T
    print("unfolded data size is",new_data.shape)
    return new_data


def reg_err(data,y,testSize):
    print(data.shape)
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = testSize,shuffle=False, random_state = 42)
    # print(X_test==data[-500:,:])
    # print(X_test.shape==data[-500:,:].shape)

    linear_regressor = LinearRegression()
    linear_regressor.fit(X_train, y_train)
    y_pred = linear_regressor.predict(X_test)
    linear_mae=mean_absolute_error(y_test,y_pred)
    linear_mse=mean_squared_error(y_test,y_pred)
    print("linear regression")
    print("mae ={:.2f}".format(linear_mae))
    print("mse ={:.2f}".format(linear_mse))

    xgb_regressor = xg.XGBRegressor()
    xgb_regressor.fit(X_train, y_train)
    y_pred = xgb_regressor.predict(X_test)
    xgb_mae=mean_absolute_error(y_test,y_pred)
    xgb_mse=mean_squared_error(y_test,y_pred)
    print("xgboost regression")
    print("mae ={:.2f}".format(xgb_mae))
    print("mse ={:.2f}".format(xgb_mse))


def energy(core,tol_u1,tol_u2):
    ''' calculating energy of u1 and u2 (the first and second factor of hosvd decomposition)
    using core tensor 
    output:
    num_comp= number of features which meet the energy tol (tol_u1) condition 
    reduced_window_size = size of window which meet the tol_u2 condition'''

    # norm of each slice through first order
    norm_arr=[np.linalg.norm(core[i,:,:]) for i in range(core.shape[0])]
    # energy list of u1
    e_u1=[sum(norm_arr[:i])/sum(norm_arr) for i in range(len(norm_arr))]
    # print("u1_energy=",e_u1)
    # select first energy which meet the condition
    reduced_window_size=next(e_u1.index(x) for x in e_u1 if x>=tol_u1) +1

    # norm of each slice through second order
    norm_arr=[np.linalg.norm(core[:,i,:]) for i in range(core.shape[1])]
    #energy list of U2 (features)
    e_u2=[sum(norm_arr[:i])/sum(norm_arr)  for i in range(len(norm_arr))]
    # print("u2_energy=",e_u2)
    # select first energy which meet the condition
    num_comp=next(e_u2.index(x) for x in e_u2 if x>=tol_u2) +1 

    return reduced_window_size,num_comp


In [19]:
# import timeseries data
dateparse = lambda x: pd.to_datetime(x, format = "%Y-%m-%d %H:00:00", errors='coerce')
data= pd.read_csv('..\kaggle dataset\dataframes.csv' ,index_col=["datetime"], parse_dates=['datetime'], date_parser=dateparse, skipinitialspace=True)
print("original data size is",data.shape)
# select a subsample of data
data=data.iloc[:2000]
print("subsample size is",data.shape)


original data size is (46968, 11)
subsample size is (2000, 11)


In [20]:
window_size=50  
testSize=500 #for test train split
tol_u1=.9
tol_u2=.98

In [21]:
tensor_data=tensorizing(window_size,data)
# full rank hosvd (core is used for energy calculation)
core,_=hosvd(tensor_data)
#optimized number of reduced_window_size,num_comp based on energy
reduced_window_size, num_comp = energy(core,tol_u1,tol_u2)
# reduced_window_size,num_comp=20,5
print("reduced_window_size=",reduced_window_size," num_comp=",num_comp)
# 2D dataset based on reduced hosvd 
new_data=data_reduced(tensor_data,reduced_window_size,num_comp)


tensorized data shape is (50, 10, 1951)


c:\Users\User.MORADI-PC\anaconda3\envs\tf\lib\site-packages\tensorly\decomposition\_tucker.py:60: Warning: No value given for 'rank'. The decomposition will preserve the original size.
  warnings.warn(message, Warning)


reduced_window_size= 10  num_comp= 4
reduced data size is (10, 4, 1951)
unfolded data size is (1951, 40)


In [22]:
y_newdata=data.iloc[window_size-1:,10]
print ("new method errors:")
reg_err(new_data,y_newdata,testSize)

new method errors:
(1951, 40)
linear regression
mae =61.67
mse =7226.89
xgboost regression
mae =62.34
mse =6929.85


In [23]:
no_target_data=data.drop(columns=['DEMAND'])
pca = PCA(n_components=num_comp).fit(no_target_data)
X=pca.transform(no_target_data)
y_pca=data.iloc[:,10]
print ("pca method errors:")
reg_err(X,y_pca,testSize)

pca method errors:
(2000, 4)
linear regression
mae =50.53
mse =5874.98
xgboost regression
mae =68.37
mse =9148.83
